

Estimated time needed: **60** minutes.

## Introduction


1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for prettytable, since package 'wheel' is not installed.
    Running setup.py install for prettytable: started
    Running setup.py install for prettytable: finished with status 'done'


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql



### Task 1

##### Display the names of the unique launch sites  in the space mission


In [10]:
%sql select distinct LAUNCH_SITE from SPACEXTBL

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


launch_site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [12]:
%sql select * from SPACEXTBL where LAUNCH_SITE like 'CCA%' limit 5

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


DATE,time__utc_,booster_version,launch_site,payload,payload_mass__kg_,orbit,customer,mission_outcome,landing__outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [13]:
%sql select SUM(PAYLOAD_MASS__KG_) from SPACEXTBL where CUSTOMER = 'NASA (CRS)'

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


1
45596


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [14]:
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTBL where booster_version like 'F9 v1.1%'

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


1
2534


### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [16]:
%sql select MIN(DATE) from SPACEXTBL where LANDING__OUTCOME = 'Success (ground pad)'

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


1
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [11]:
%sql select BOOSTER_VERSION, PAYLOAD_MASS__KG_ from SPACEXTBL where LANDING__OUTCOME = 'Success (drone ship)' AND \
            PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


booster_version,payload_mass__kg_
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


### Task 7

##### List the total number of successful and failure mission outcomes


In [25]:
%sql SELECT * FROM (SELECT Count(Mission_Outcome) AS Success FROM SPACEXTBL WHERE Mission_Outcome LIKE '%Success%'), \
(SELECT Count( Mission_Outcome) AS Failure FROM SPACEXTBL WHERE Mission_Outcome LIKE '%Failure%' ) 

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


success,failure
100,1


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [29]:
%sql SELECT Booster_Version AS Booster_ve FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ IN \
(SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


booster_ve
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [45]:
%sql SELECT landing__outcome, booster_version, launch_site \
FROM SPACEXTBL WHERE landing__outcome LIKE 'Failure (drone ship)' AND DATE LIKE '2015%'

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


landing__outcome,booster_version,launch_site
Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [49]:
%sql SELECT landing__outcome, count(landing__outcome) AS countl FROM SPACEXTBL \
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' \
GROUP BY landing__outcome ORDER BY countl DESC

 * ibm_db_sa://chy61233:***@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/BLUDB
Done.


landing__outcome,countl
No attempt,10
Failure (drone ship),5
Success (drone ship),5
Controlled (ocean),3
Success (ground pad),3
Failure (parachute),2
Uncontrolled (ocean),2
Precluded (drone ship),1


## Thank You!
